In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
class C2W(nn.Module):
    def __init__(self, config):
        super(C2W, self).__init__()
        self.char_hidden_size = config.char_hidden_size
        self.word_embed_size = config.word_embed_size
        self.lm_hidden_size = config.lm_hidden_size
        self.character_embedding = nn.Embedding(config.n_chars,config.char_embed_size) # 字符嵌入层
        self.sentence_length = config.max_sentence_length
        self.char_lstm = nn.LSTM(input_size=config.char_embed_size,hidden_size=config.char_hidden_size,
                            bidirectional=True,batch_first=True)  # 字符lstm
        self.lm_lstm = nn.LSTM(input_size=self.word_embed_size,hidden_size=config.lm_hidden_size,batch_first=True) # 语言模型lstm
        self.fc_1 = nn.Linear(2*config.char_hidden_size,config.word_embed_size) # 线性组合生成词表示
        self.fc_2 =nn.Linear(config.lm_hidden_size,config.vocab_size) # 生成类别用于预测

    def forward(self, x):
        x = torch.Tensor(x).long()
        input = self.character_embedding(x)
        char_lstm_result = self.char_lstm(input)
        word_input = torch.cat([char_lstm_result[0][:,-1,0:self.char_hidden_size],
                                char_lstm_result[0][:,0,self.char_hidden_size:]],dim=1)
        word_input = self.fc_1(word_input)
        word_input = word_input.view([-1,self.sentence_length,self.word_embed_size])
        lm_lstm_result = self.lm_lstm(word_input)[0].contiguous()
        lm_lstm_result = lm_lstm_result.view([-1,self.lm_hidden_size])
        out = self.fc_2(lm_lstm_result)
        return out

In [3]:
class config:
    def __init__(self):
        self.n_chars = 64  # 字符的个数
        self.char_embed_size = 50 # 字符嵌入大小
        self.max_sentence_length = 8 # 最大句子长度
        self.char_hidden_size = 50 # 字符lstm的隐藏层神经元个数
        self.lm_hidden_size = 150 # 语言模型的隐藏神经元个数
        self.word_embed_size = 50 # 生成的词表示大小
        config.vocab_size = 1000 # 词表大小

In [4]:
config = config()
c2w = C2W(config)
test = np.zeros([64,16])
out = c2w(test)

In [5]:
out

tensor([[ 0.0579,  0.0313,  0.0452,  ...,  0.0434,  0.0060,  0.0482],
        [ 0.0608,  0.0404,  0.0534,  ...,  0.0484,  0.0029,  0.0561],
        [ 0.0620,  0.0451,  0.0585,  ...,  0.0512,  0.0013,  0.0591],
        ...,
        [ 0.0627,  0.0495,  0.0645,  ...,  0.0535, -0.0006,  0.0602],
        [ 0.0627,  0.0499,  0.0651,  ...,  0.0536, -0.0008,  0.0600],
        [ 0.0626,  0.0501,  0.0654,  ...,  0.0536, -0.0010,  0.0599]],
       grad_fn=<AddmmBackward0>)

In [6]:
out.shape

torch.Size([64, 1000])

In [7]:
print (c2w)

C2W(
  (character_embedding): Embedding(64, 50)
  (char_lstm): LSTM(50, 50, batch_first=True, bidirectional=True)
  (lm_lstm): LSTM(50, 150, batch_first=True)
  (fc_1): Linear(in_features=100, out_features=50, bias=True)
  (fc_2): Linear(in_features=150, out_features=1000, bias=True)
)
